In [1]:
from scraper.scrape_pipeline import ScrapePipeline
from database.database_pipeline import DataManager
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
from explainer.explainer import Explainer
from models.sentiment_analysis_model import SentimentAnalysisModel
from models.topic_model import TopicModel

c:\Users\redbl\Desktop\Courseworks\Y3S2\DSA3101 Data Science in Practice\Project\Banking-Sentiment-Analysis\server\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\redbl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
scraper = ScrapePipeline(DataManager("waddlewafflesbackend", "UJVkE6GCHQrokIhM"))

In [10]:
all_reviews, playstore_scraped_datetime, app_scraped_datetime = scraper.scrape_new_reviews()

In [11]:
all_reviews

,review,rating,bank,title,source,year,month,day
0,i like how the app is really light and fast co...,5,GXS,,playstore,2024,4,15
1,cannot add card to google pay also cannot add ...,1,GXS,,playstore,2024,4,13
2,generally good however please add either nfc f...,4,GXS,,playstore,2024,4,13
3,with continuous failed to log in contacted cso...,1,GXS,,playstore,2024,4,11
4,i don t get the negative reviews here i instal...,5,GXS,,playstore,2024,4,9
...,...,...,...,...,...,...,...,...
1241,trust bank sg is one of the best app i used an...,5,TRUST,,playstore,2024,3,18
1242,trust bank singapore is a digital bank backed ...,5,TRUST,,playstore,2024,3,17
1243,super easy with applying a new account credit ...,4,TRUST,,playstore,2024,3,17
1244,service to unlock the card where apps already ...,4,TRUST,,playstore,2024,3,16


In [3]:
app_scraped_datetime

datetime.datetime(2024, 4, 16, 2, 27, 36, 974202)

In [7]:
playstore_scraped_datetime.strftime("%Y-%m-%d %H:%M:%S")

'2024-04-16 02:22:06'

In [3]:
sentiment_model = SentimentAnalysisModel()
explainer = Explainer(sentiment_model.pipe)
topic_model = TopicModel(["login", "interface", "stability", "update", "notifications", "speed", "service", "functions", "security"])

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
k = pd.read_csv("newdata.csv")
del k["Unnamed: 0"]

In [5]:
print(".............RUNNING SENTIMENT MODEL..............")
sentiments = sentiment_model.get_sentiment(k["review"].to_list())
k["sentiment"] = [max(sentiment, key=sentiment.get) for sentiment in sentiments]

.............RUNNING SENTIMENT MODEL..............


In [17]:
for i in k.iloc:
    if i["bank"] == "GXS":
        print(i["review"])

i like how the app is really light and fast compared to other bank apps
cannot add card to google pay also cannot add card to touch n go 
generally good however please add either nfc function to the app or allow link to google wallet or apple pay everyone is going wallet less these days without having the need to carry physical wallets or cards and why are we unable to scan sgqr codes for payments only paynow qr codes would definitely get my stars if not for the above shortcomings stars for now 
with continuous failed to log in contacted cso to terminate the account and we re informed once acct close can t be reinstated im fine with it since the problem it s on your app 
great banking app with attractive interest rates please allow us to add and or save payees so we dont have to keep typing out uen numbers or account numbers would be nice to be able to add the debit card to apple pay too 
very smooth to use and attractive app simple features and excited to see more advanced banking fea

In [15]:
k[(k["year"]==2024)& (k["month"]==4)&(k["day"]==14)&(k["bank"]=="GXS")]

,review,rating,bank,title,source,year,month,day,sentiment,topic


In [ ]:
print(".............RUNNING TOPIC MODEL..............")
topics = topic_model.get_topic(k["review"].to_list())
k["topic"] = topics["Predicted Topic"]

In [ ]:
print(".............RUNNING EXPLAINER MODEL..............")
associations = [list(map(lambda x:list(dict.fromkeys(x))[:3], explainer.get_keywords(k.iloc[i]["review"], k.iloc[i]["sentiment"]))) for i in range(k.shape[0])]
k["associations"] = associations

In [22]:
j = pd.read_csv("stuff.csv")

In [37]:
import ast

In [42]:
j

,review,rating,bank,title,source,year,month,day,sentiment,topic,associations
0,i like how the app is really light and fast co...,5,GXS,NaN,playstore,2024,4,15,Positive,speed,"[like, apps, really]"
1,cannot add card to google pay also cannot add ...,1,GXS,NaN,playstore,2024,4,13,Negative,functions,"[add, google, go]"
2,generally good however please add either nfc f...,4,GXS,NaN,playstore,2024,4,13,Neutral,functions,"[unable, function, would]"
3,with continuous failed to log in contacted cso...,1,GXS,NaN,playstore,2024,4,11,Negative,login,"[failed, problem, terminate]"
4,easy to use,5,DBS,NaN,playstore,2024,4,14,Positive,interface,"[easy, use]"
...,...,...,...,...,...,...,...,...,...,...,...
147,tried reinstall restart re adding but to no av...,1,Trust,unable to add credit card to apple wallet,appstore,2024,3,25,Neutral,update,"[tried, restart, install]"
148,great,3,Trust,great,appstore,2024,3,7,Positive,update,[great]
149,thank you,5,Trust,convenient happy with tye service,appstore,2024,4,7,Positive,update,[thank]
150,i bought packs of toilet rolls pack of eggs pl...,5,Trust,just in time,appstore,2024,3,29,Neutral,functions,"[pack, coins, bought]"


In [49]:
new[new["day"]>14]

,review,rating,bank,title,source,year,month,day,sentiment,topic,associations
0,i like how the app is really light and fast co...,5,GXS,NaN,playstore,2024,4,15,Positive,speed,"[like, apps, really]"


In [34]:
data_manager = DataManager("waddlewafflesbackend", "UJVkE6GCHQrokIhM")

In [51]:

data_manager.update_miscellaneous("playstore", "datetime", {"$set": {"latestdate": "2024-04-15 23:50:00"}})
data_manager.update_miscellaneous("appstore", "datetime", {"$set": {"latestdate": "2024-04-15 23:50:00"}})

Timestamp('2024-04-10 23:42:07')